# Parameter and Distribution Discovery

This notebook helps to discover the correct distributions, and parameters for these distributions, to best approximate your real-world system in the model. 

As the model relies on random sampling of realistic values, ensuring that it knows what a 'realistic value' is is crucial. 

This notebook helps to generate distributions for 
- inter-arrival times (in hours and out of hours)
- length of stay (subset by modified rankin scale (MRS) score on presentation and stroke type)
- MRS score on presentation

## Data Path Setup

### LOS Data

Please pass in a .csv file with the following columns: 

- Diagnosis *(text label or integer values)*
- MRS *(integer values between 0 and 6)*
- LOS *(integer number of days)*

### IAT setup

Please pass in a .csv file with the following columns: 

- in-hours *(integer number of minutes between arrivals)*
- out-of-hours *(integer number of minutes between arrivals)*

In [ ]:
# UPDATE THIS IF YOUR FILE IS CALLED SOMETHING DIFFERENT OR STORED SOMEWHERE ELSE
los_csv_file_path = "../input_data/los.csv"
iat_csv_file_path = "../input_data/iat.csv"

The following should only be adjusted by advanced users. 
These parameters affect the tolerance of the script to recommending the exponential distribution if it is a 'good enough' approximation of the provided data.

In [ ]:
ABSOLUTE_GOOD = 0.005     # below this, fits are effectively perfect
RELATIVE_TOL = 1.5      # 50% worse than best allowed

Now click 'Run All'. 

Three files will be generated in this folder providing the suggested distributions and parameters for your simulation. 

These should be used to update the
- parameters in the file src/stroke_ward_model/inputs.py
- distributions in the file src/stroke_ward_model/distributions.py

In [ ]:
import pandas as pd
from distfit import distfit
import plotly.express as px

# Length of Stay Processing

In [ ]:
los_df = pd.read_csv(los_csv_file_path)

results = []
plots = []

for diagnosis in los_df["diagnosis"].unique():
    los_df_diagnosis = los_df[los_df["diagnosis"]==diagnosis]

    fig = px.histogram(los_df_diagnosis.sort_values('mrs'),x="los", facet_col="mrs",title=diagnosis)
    fig.update_yaxes(matches=None, showticklabels=True)

    plots.append(fig)

    for mrs in los_df_diagnosis["mrs"].unique():
        los_df_diagnosis_mrs = los_df_diagnosis[los_df_diagnosis["mrs"]==mrs]

        # Limit these to key distributions available in the sim-tools package
        dfit = distfit(todf=True, distr=['norm', 'expon', 'pareto', 'weibull', 'gamma', 'lognorm', 'beta'])

        # Search for best theoretical fit on your empirical data
        model_results = dfit.fit_transform((los_df_diagnosis_mrs["los"].dropna().values))

        summary = dfit.summary.sort_values("score")

        best = summary.iloc[0]
        best_score = best["score"]

        # tolerance rule
        tolerance = 1.3  # 30% worse than best is acceptable

        exp_row = summary[summary["name"] == "expon"]


        if not exp_row.empty:
            exp_score = exp_row.iloc[0]["score"]

            if best_score < ABSOLUTE_GOOD:
                # Fits are indistinguishable → prefer exponential
                chosen = exp_row.iloc[0]

            elif exp_score <= RELATIVE_TOL * best_score:
                chosen = exp_row.iloc[0]

            else:
                chosen = best
        else:
            chosen = best

        results.append({
            "diagnosis": diagnosis,
            "mrs": mrs,
            "best_distribution": chosen["name"],
            "params": chosen["params"],
            "loc": chosen["loc"],
            "scale": chosen["scale"],
            "exp_score": exp_row.iloc[0]["score"],
            "exp_score_diff_from_best": exp_score - best_score,
            "score": chosen["score"],
            "best_score": best_score,
            "score_ratio": chosen["score"] / best_score,
            "n_datapoints_fitted_on": len(los_df_diagnosis_mrs),
            "min_los_observed": los_df_diagnosis_mrs["los"].min(),
            "median_los_observed": los_df_diagnosis_mrs["los"].median(),
            "mean_los_observed": round(los_df_diagnosis_mrs["los"].mean(),2),
            "max_los_observed": los_df_diagnosis_mrs["los"].max()

        })

results_df = pd.DataFrame(results).sort_values(["diagnosis", "mrs"])
results_df["shape1"] = results_df["params"].apply(lambda x: x[0] if len(x) > 0 else None)
results_df["shape2"] = results_df["params"].apply(lambda x: x[1] if len(x) > 1 else None)

results_df.to_csv("los_params.csv", index=False)

results_df

In [ ]:
[x.show() for x in plots]

# For MRS

In [ ]:
import plotly.figure_factory as ff

ich = los_df[los_df["diagnosis"]=="ICH"]["mrs"].values
i = los_df[los_df["diagnosis"]=="I"]["mrs"].values

fig = ff.create_distplot([ich,i], group_labels=["ICH","I"])

fig

In [ ]:
results = []

for diagnosis in los_df["diagnosis"].unique():

    los_df_diagnosis = los_df[los_df["diagnosis"]==diagnosis]

    values = los_df_diagnosis["mrs"].dropna().astype(int)

    pmf = (
        values
        .value_counts(normalize=True)
        .reindex(range(6), fill_value=0.0)
    )

    results.append({
        "diagnosis": diagnosis,
        "mrs": mrs,
        "distribution": "categorical",
        "pmf": pmf.to_dict(),
        "n_datapoints_fitted_on": len(los_df_diagnosis),
    })

results_df = pd.DataFrame(results)

pmf_df = (
    results_df["pmf"]
    .apply(pd.Series)
    .rename(columns=lambda x: f"mrs_{int(x)}_prob")
)

results_df = pd.concat([results_df.drop(columns=["pmf"]), pmf_df], axis=1)
results_df.to_csv("mrs_params.csv")
results_df

Repeat without stratifying by diagnosis:

In [ ]:
results = []

values = los_df["mrs"].dropna().astype(int)

pmf = (
    values
    .value_counts(normalize=True)
    .reindex(range(6), fill_value=0.0)
)

results.append({
    "diagnosis": diagnosis,
    "mrs": mrs,
    "distribution": "categorical",
    "pmf": pmf.to_dict(),
    "n_datapoints_fitted_on": len(los_df_diagnosis),
})

results_df = pd.DataFrame(results)

pmf_df = (
    results_df["pmf"]
    .apply(pd.Series)
    .rename(columns=lambda x: f"mrs_{int(x)}_prob")
)

results_df = pd.concat([results_df.drop(columns=["pmf"]), pmf_df], axis=1)
results_df

# For Inter-arrival times

In [ ]:
iat_df = pd.read_csv(iat_csv_file_path)

results = []
plots = []

for time_period in ["in-hours","out-of-hours"]:
    time_period_df = iat_df[[time_period]].dropna()

    fig = px.histogram(time_period_df, title=time_period)

    plots.append(fig)

    dfit = distfit(todf=True, distr=['norm', 'expon', 'pareto', 'weibull', 'gamma', 'lognorm', 'beta'])

    # Search for best theoretical fit on your empirical data
    model_results = dfit.fit_transform((time_period_df[time_period].dropna().values))

    summary = dfit.summary.sort_values("score")

    best = summary.iloc[0]
    best_score = best["score"]

    # tolerance rule
    tolerance = 1.3  # 30% worse than best is acceptable

    exp_row = summary[summary["name"] == "expon"]

    if not exp_row.empty:
        exp_score = exp_row.iloc[0]["score"]

        if best_score < ABSOLUTE_GOOD:
            # Fits are indistinguishable → prefer exponential
            chosen = exp_row.iloc[0]

        elif exp_score <= RELATIVE_TOL * best_score:
            chosen = exp_row.iloc[0]

        else:
            chosen = best
    else:
        chosen = best


    results.append({
        "time_period": time_period,
        "best_distribution": chosen["name"],
        "params": chosen["params"],
        "loc": chosen["loc"],
        "scale": chosen["scale"],
        "exp_score": exp_row.iloc[0]["score"],
        "exp_score_diff_from_best": exp_score - best_score,
        "score": chosen["score"],
        "best_score": best_score,
        "score_ratio": chosen["score"] / best_score,
        "n_datapoints_fitted_on": len(time_period_df),
        "min_iat_observed": time_period_df[time_period].min(),
        "median_iat_observed": time_period_df[time_period].median(),
        "mean_iat_observed": time_period_df[time_period].mean(),
        "max_iat_observed": time_period_df[time_period].max()

    })

results_df = pd.DataFrame(results)

results_df.to_csv("iat_params.csv", index=False)

results_df

In [ ]:
[x.show() for x in plots]